# 📘 Section 0. 통계적 유의성이란?

---

### 💡 핵심 개념
- **유의성(Significance)**: 두 집단 간의 차이가 **우연이 아니라 통계적으로 의미 있는지** 평가  
- “눈으로 보기엔 달라 보이지만, 정말로 ‘통계적으로’ 다를까?”를 검정  

---

### 🎯 학습 목표
| 구분 | 내용 | 주요 함수 |
|------|------|-----------|
| 1️⃣ | 독립표본 t-검정 | `stats.ttest_ind()` |
| 2️⃣ | 대응표본 t-검정 | `stats.ttest_rel()` |
| 3️⃣ | 비모수 검정 (Mann-Whitney / Wilcoxon) | `mannwhitneyu()`, `wilcoxon()` |
| 4️⃣ | 범주형 검정 (카이제곱 / 피셔) | `chi2_contingency()`, `fisher_exact()` |

---

### ⚖️ 기본 해석 기준
| p-value | 해석 |
|----------|------|
| p < 0.05 | 유의함 (차이가 ‘있다’) |
| p ≥ 0.05 | 유의하지 않음 (차이가 ‘없다’) |

# 🧠 Section 1. 독립표본 t-검정

---

### 💡 목적
서로 다른 두 집단의 평균 차이를 비교  
예: “A반과 B반의 평균 키가 다를까?”

---

### ⚙️ 가정 조건
- 두 그룹은 서로 독립  
- 각 그룹 데이터는 **정규분포**를 따름 (`stats.shapiro`)  
- 두 그룹의 분산이 동일 (`stats.levene`)  

---

### 🎯 귀무가설(H₀)
> 두 집단의 평균은 같다.

대립가설(H₁): 두 집단의 평균은 다르다.

---

### 📚 t-검정 요약

| 구분 | 내용 |
|------|------|
| 🎯 목적 | 두 독립 집단의 평균 차이 검정 |
| 📋 가정 | ① 독립성 ② 정규성 ③ 등분산성 |
| 🔍 귀무가설(H₀) | 두 집단의 평균은 같다 |
| 🧠 대립가설(H₁) | 두 집단의 평균은 다르다 |
| 📈 해석 | p < 0.05 → 통계적으로 유의한 차이 있음 |

---

### 🧩 응용 아이디어
- 반별 평균 키 비교 (A반 vs B반)
- 남녀 평균 시험 점수 비교
- 커피 마신 그룹 vs 안 마신 그룹의 집중도 비교  
- 학과별 수면시간 차이 검정 등

In [ ]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np

# ==========================================================
# 🎲 01. 데이터 생성
# ==========================================================
# - 두 개의 독립된 그룹(A, B)을 만듭니다.
# - 각 그룹은 서로 다른 평균을 가지며(170cm vs 173cm),
#   학생들이 직접 평균, 표준편차, 표본 수를 바꿔보며 실험할 수 있습니다.

np.random.seed(123)  # 재현 가능성을 위해 난수 고정

group_A = np.random.normal(170, 5, 30)  # 평균 170, 표준편차 5, 표본 30개
group_B = np.random.normal(173, 5, 30)  # 평균 173, 표준편차 5, 표본 30개

# 두 그룹을 합쳐서 하나의 DataFrame으로 생성
df = pd.DataFrame({
    'group': ['A']*30 + ['B']*30,
    'height': np.concatenate([group_A, group_B])
})

In [ ]:
# ==========================================================
# 📊 02. 기초 통계 확인
# ==========================================================
print("그룹별 평균 높이 (단위: cm):")
print(df.groupby('group')['height'].mean())  # 그룹별 평균

# 박스플롯으로 시각화
# → 각 그룹의 분포를 시각적으로 비교
df.boxplot(column='height', by='group', grid=False, figsize=(5,4))
plt.title("A vs B Group Heights")
plt.suptitle("")  # 자동 제목 제거
plt.ylabel("Height (cm)")
plt.show()

In [ ]:
# ==========================================================
# 🧠 03. 정규성 검정 (Shapiro-Wilk Test)
# ==========================================================
# - t-검정을 하기 전, 각 집단이 "정규분포"를 따르는지 확인
# - 귀무가설(H0): 데이터는 정규분포를 따른다.
# - p-value < 0.05 → 정규분포 아님 → 비모수 검정 필요 (Mann–Whitney)
# - p-value ≥ 0.05 → 정규분포 가정 OK → t-test 가능

print("Shapiro-Wilk 정규성 검정:")
print("Group A:", stats.shapiro(group_A))
print("Group B:", stats.shapiro(group_B))

In [ ]:
# ==========================================================
# ⚖️ 04. 등분산성 검정 (Levene’s Test)
# ==========================================================
# - 두 그룹의 분산이 같은지 확인
# - 귀무가설(H0): 두 그룹의 분산이 동일하다.
# - p-value < 0.05 → 분산이 다름 (equal_var=False)
# - p-value ≥ 0.05 → 분산 동일 (equal_var=True)

print("Levene 등분산 검정:")
print(stats.levene(group_A, group_B))

In [ ]:
# ==========================================================
# 🎯 05. 독립표본 t-검정 (Independent t-test)
# ==========================================================
# - 두 독립된 그룹 간 평균 차이가 유의한가 검정
# - 귀무가설(H0): 두 그룹의 평균이 같다.
# - 대립가설(H1): 두 그룹의 평균이 다르다.
# - p-value < 0.05 → 평균 차이가 통계적으로 유의함

result = stats.ttest_ind(group_A, group_B, equal_var=True)
print("독립표본 t-검정 결과:", result)

In [ ]:
# ==========================================================
# 📈 06. 해석 가이드
# ==========================================================
# t통계량(t-statistic): 두 평균의 차이가 얼마나 큰지를 나타냄 (절댓값이 클수록 차이가 큼)
# p-value: 차이가 ‘우연일 확률’
#    → p < 0.05 이면 "A와 B는 평균적으로 유의미하게 다르다" 라고 판단
# ==========================================================

# 예시 출력
# Shapiro-Wilk: 두 그룹 모두 p>0.05 → 정규분포 가정 OK
# Levene: p>0.05 → 분산 동일
# t-test: p<0.05 → 평균 차이 유의미함 (A보다 B가 평균적으로 큼)

# 🔁 Section 2. 대응표본 t-검정

---

### 💡 목적
같은 집단의 **전후(before–after)** 차이를 비교  
예: “영양제를 복용하기 전과 후의 체력 점수가 달라졌을까?”

---

### ⚙️ 가정 조건
- 같은 사람(또는 동일 대상)을 두 번 측정  
- 두 값의 차이가 정규분포를 따른다. (`stats.shapiro(after-before)`)

---

### 🎯 귀무가설(H₀)
> 두 시점의 평균은 같다.

---

### 🧾 특징 요약

| 구분 | 설명 |
|------|------|
| 🎯 **비교 대상** | 동일 집단의 ‘전(before)’ vs ‘후(after)’ |
| 🧩 **데이터 구조** | 두 열 (before, after) |
| 📈 **검정 목적** | 평균 차이가 우연이 아닌 통계적으로 유의한가 |
| ⚙️ **가정 조건** | (after - before)의 분포가 정규성을 띰 |
| 🔍 **귀무가설 (H₀)** | 두 시점의 평균은 같다 |
| 💥 **대립가설 (H₁)** | 두 시점의 평균은 다르다 |
| 📊 **사용 함수** | `stats.ttest_rel(before, after)` |

---

### 🧠 해석 가이드

| 통계량 | 의미 |
|---------|------|
| **t-statistic** | 두 시점 평균 차이의 크기 |
| **p-value** | 그 차이가 우연일 확률 |
| **결론** | p < 0.05 → “유의한 차이 있음” |

---

### 📚 예시 해석

> 운동 전 평균: 75.2  
> 운동 후 평균: 77.8  
> 평균 증가: +2.6  
> p-value = 0.01 → **운동 후 점수가 유의하게 향상됨!**

In [ ]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np

# ==========================================================
# 🎲 01. 데이터 생성
# ==========================================================
# - 같은 집단을 'Before', 'After'로 두 번 측정한 상황을 가정합니다.
# - 예: 같은 학생 20명의 운동 전후 체력 점수

np.random.seed(123)  # 랜덤 고정 (매번 같은 결과 재현 가능)

# 'before' 점수는 평균 75, 표준편차 5인 정규분포에서 생성
before = np.random.normal(75, 5, 20)

# 'after' 점수는 'before'보다 평균적으로 약 2점 상승 (±3의 오차 포함)
after = before + np.random.normal(2, 3, 20)

# 데이터프레임 구성
df = pd.DataFrame({'before': before, 'after': after})

In [ ]:
# ==========================================================
# 📊 02. 기초 통계 요약
# ==========================================================
print("전/후 평균 비교:")
print(df.mean())  # before와 after의 평균값 비교

# 평균 변화량 계산 (after - before)
print("평균 변화량:", (df['after'] - df['before']).mean())

In [ ]:
# ==========================================================
# 🎨 03. 시각화 (박스플롯)
# ==========================================================
# - 각 시점의 분포를 시각적으로 비교
# - 변화의 방향과 이상치(outlier) 확인에 유용

fig, axes = plt.subplots(1, 2, figsize=(8,4))

df['before'].plot.box(grid=False, ax=axes[0], title='Before')
df['after'].plot.box(grid=False, ax=axes[1], title='After')

plt.suptitle("Comparison of Fitness Scores Before and After Exercise", fontsize=13)
plt.show()

In [ ]:
# ==========================================================
# 🧠 04. 정규성 검정 (Shapiro–Wilk Test)
# ==========================================================
# - 대응표본 t-검정에서는 '차이(after-before)'가 정규분포를 따라야 함
# - 귀무가설(H0): 차이 값은 정규분포를 따른다
# - p-value < 0.05 → 정규분포 아님 → 비모수 검정(Wilcoxon) 고려
# - p-value ≥ 0.05 → 정규분포 가정 OK → t-test 가능

print("Shapiro 정규성 검정 (after-before):")
shapiro_result = stats.shapiro(df['after'] - df['before'])
print(shapiro_result)

In [ ]:
# ==========================================================
# 🎯 05. 대응표본 t-검정 (Paired t-test)
# ==========================================================
# - 두 시점(before, after)의 평균이 통계적으로 다른지 비교
# - 귀무가설(H0): 두 시점의 평균은 같다
# - 대립가설(H1): 두 시점의 평균은 다르다
# - p-value < 0.05 → 평균 차이가 유의미함

result = stats.ttest_rel(df['before'], df['after'])
print("대응표본 t-검정 결과:", result)

In [ ]:
# ==========================================================
# 🧩 06. 해석 가이드
# ==========================================================
# t통계량(t-statistic): 차이의 크기 / 표준오차
# p-value: 차이가 우연일 확률
#    → p < 0.05 → "운동 후 점수가 통계적으로 유의하게 증가했다"고 해석
# ==========================================================

# 💬 예시 출력
# Shapiro: p>0.05 → 정규성 OK
# t-test: p<0.05 → 전후 점수 차이가 유의함

# 🧮 Section 3. 비모수 검정(Non-parametric Test)

---

### 💡 개념 정리
- t-검정은 **정규분포를 전제**로 하는데, 실제 데이터는 종종 **비대칭적**이거나 **이상치**가 많음.  
- 이런 경우 평균 대신 **순위(rank)**를 사용하여 비교하는 것이 **비모수 검정**입니다.

---

| 검정명 | 데이터 형태 | 비교 대상 | 주요 함수 | 예시 |
|---------|-------------|------------|-------------|-------|
| **Mann–Whitney U test** | 독립된 두 집단 | 평균이 아닌 분포(중앙값) | `stats.mannwhitneyu()` | 남녀 스트레스 점수 비교 |
| **Wilcoxon Signed-Rank test** | 동일 집단의 전후 비교 | 쌍(pair) 데이터 차이 | `stats.wilcoxon()` | 치료 전후 혈압 비교 |

---

### 🧠 비모수 검정이 필요한 상황
- 데이터가 정규분포를 따르지 않음 (p < 0.05 in Shapiro test)
- 이상치(outlier)가 많음
- 순위형(ordinal) 데이터 (예: 만족도 등급 1~5점)

---

### 🎯 핵심 해석
- 귀무가설(H₀): 두 집단의 분포(또는 중앙값)는 같다.  
- 대립가설(H₁): 두 집단의 분포가 다르다.
- p-value < 0.05 → 두 집단은 통계적으로 유의하게 다름.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# ==========================================================
# 🎲 01. 예시 데이터 생성 (정규분포 X, 범위 다름)
# ==========================================================
# - 그룹 A: 점수 50~80 사이에서 균등 분포
# - 그룹 B: 점수 60~90 사이에서 균등 분포
# - 즉, 두 그룹은 평균은 다르지만 분포가 정규가 아님

np.random.seed(123)
group_A = np.random.uniform(50, 80, 20)
group_B = np.random.uniform(60, 90, 20)

In [ ]:
# ==========================================================
# 📊 02. 기초 통계 확인
# ==========================================================
print("그룹별 평균:")
print(f"A 평균 = {np.mean(group_A):.2f}, B 평균 = {np.mean(group_B):.2f}")

# 박스플롯으로 시각화 → 분포 차이 직관 확인
plt.boxplot([group_A, group_B], labels=['A', 'B'])
plt.title("Mann–Whitney U Test Example")
plt.ylabel("Score")
plt.show()

In [ ]:
# ==========================================================
# 🎯 03. Mann–Whitney U 검정 수행
# ==========================================================
# - 귀무가설(H₀): 두 집단의 분포가 같다.
# - 대립가설(H₁): 두 집단의 분포가 다르다.

result = stats.mannwhitneyu(group_A, group_B)
print("Mann–Whitney U Test 결과:")
print(result)

In [ ]:
# ==========================================================
# 🧩 04. 해석
# ==========================================================
# U통계량: 순위 기반 차이 측정
# p-value < 0.05 → A와 B의 분포가 통계적으로 유의하게 다름

### 💬 Mann–Whitney U Test 해석 예시

| 항목 | 결과 | 해석 |
|------|------|------|
| U통계량 | 100.5 | 두 그룹 순위 차이 |
| p-value | 0.02 | 5%보다 작음 → 유의함 |
| 결론 | A와 B의 중앙값이 통계적으로 다름 |

---

> **예시 해석:**  
> “그룹 B의 점수가 전반적으로 그룹 A보다 높게 나타남.”  
> → 데이터가 정규분포가 아니므로 평균 대신 **순위 차이**로 판단.

### 🔁 Wilcoxon Signed-Rank 검정 (대응집단 비교)

In [ ]:
# ==========================================================
# 🎲 01. 예시 데이터 생성 (대응집단)
# ==========================================================
# - 동일한 사람의 Pre/Post 테스트 점수 비교
# - 예: 학습 전후 성취도, 치료 전후 혈압 등

pre = np.random.randint(60, 85, 15)  # 사전 점수
post = pre + np.random.randint(-3, 6, 15)  # 사후 점수 (± 변화 포함)

In [ ]:
# ==========================================================
# 📊 02. 평균 변화 확인
# ==========================================================
print("평균 변화 (After - Before):", np.mean(post - pre))

# 시각화
plt.boxplot([pre, post], labels=['Pre', 'Post'])
plt.title("Wilcoxon Signed-Rank Test Example")
plt.ylabel("Score")
plt.show()

In [ ]:
# ==========================================================
# 🎯 03. Wilcoxon 검정 수행
# ==========================================================
# - 귀무가설(H₀): 전후 중앙값 차이가 없다.
# - 대립가설(H₁): 전후 중앙값 차이가 있다.
# - 정규성 가정 필요 없음.

result = stats.wilcoxon(pre, post)
print("Wilcoxon Test 결과:")
print(result)

In [ ]:
# ==========================================================
# 🧩 04. 해석
# ==========================================================
# T통계량: 차이의 절댓값 순위합을 기반으로 계산
# p-value < 0.05 → 전후 차이가 통계적으로 유의함

### 📘 비모수 검정 요약

| 검정 | 비교 대상 | 가정 | 함수 | 핵심 해석 |
|------|------------|------|--------|-------------|
| **Mann–Whitney U** | 독립된 두 집단 | 정규성 불필요 | `stats.mannwhitneyu()` | 순위 차이 |
| **Wilcoxon Signed-Rank** | 동일 집단의 전후 | 정규성 불필요 | `stats.wilcoxon()` | 순위 차이 |

---

### ⚖️ 해석 가이드
| p-value | 해석 |
|----------|------|
| < 0.05 | 유의함 (중앙값 차이 있음) |
| ≥ 0.05 | 유의하지 않음 (중앙값 차이 없음) |

---

### 📚 응용 아이디어
- Mann–Whitney:  
  → “A학과와 B학과의 만족도 점수가 다를까?”  
- Wilcoxon:  
  → “AI 실습 전과 후 학생의 자신감 점수가 달라졌을까?”  

---

# 🧮 Section 4. 범주형 데이터 검정 (Chi-Square & Fisher’s Exact Test)

---

### 💡 개념 요약
- **t-검정 / U-검정 / Wilcoxon**은 “수치형 데이터(평균, 중앙값)” 비교에 사용됨.
- 하지만 **성별, 합격여부, 치료 성공률**처럼 **범주형(명목형) 데이터**는 다름.
- 이런 경우 “비율(비중)이 차이 나는가?”를 보기 위해 **교차분석**을 사용함.

---

| 검정 | 데이터 형태 | 사용 함수 | 주요 목적 |
|------|--------------|-------------|-------------|
| **Chi-Square Test (카이제곱 검정)** | 표본수가 충분할 때 | `stats.chi2_contingency()` | 관찰된 빈도와 기대 빈도의 차이 검정 |
| **Fisher’s Exact Test (피셔의 정확 검정)** | 표본수가 작을 때 | `stats.fisher_exact()` | 작은 표본에서의 정확한 확률 계산 |

---

### 🎯 귀무가설(H₀)
> 두 변수(예: 성별 vs 합격여부)는 **독립적이다.**  
> 즉, 성별에 따라 합격률이 다르지 않다.

p-value < 0.05 → 귀무가설 기각 → “두 변수 간 **유의한 관련성**이 있다.”


In [ ]:
from scipy import stats
import numpy as np

# ==========================================================
# 🎲 01. 예시 데이터: 성별에 따른 시험 합격 여부
# ==========================================================
# [합격, 불합격]
men = [10, 10]       # 남성 10명 합격, 10명 불합격
women = [15, 65]     # 여성 15명 합격, 65명 불합격

In [ ]:
# ==========================================================
# 📊 02. 카이제곱 검정 수행
# ==========================================================
# 귀무가설(H₀): 성별과 합격 여부는 독립이다.
# 대립가설(H₁): 성별에 따라 합격률이 다르다.

result = stats.chi2_contingency([men, women])

# 출력 값: (카이제곱 통계량, p-value, 자유도, 기대빈도표)
print("기대빈도표(Expected Frequencies):\n", result[3])
print("\n 카이제곱 검정 결과:")
print(f"Chi-square 통계량 = {result[0]:.3f}")
print(f"p-value = {result[1]:.4f}")

In [ ]:
# ==========================================================
# 🧩 03. 해석 가이드
# ==========================================================
# 기대빈도(Expected Frequency): 성별과 관계없이 우연히 발생했을 경우의 예측 값
# p-value < 0.05 → 실제로 성별에 따라 합격률이 다르다고 볼 수 있음

### 💬 Chi-Square 해석 예시

| 항목 | 결과 | 해석 |
|------|------|------|
| 카이제곱 통계량 | 7.84 | 관찰값과 기대값의 차이 크기 |
| 자유도(df) | 1 | (행-1)*(열-1) |
| p-value | 0.005 | 0.05보다 작음 → 유의함 |
| 결론 | 성별에 따라 합격률 차이가 유의하게 존재함 |

---

✅ **즉:** “남성과 여성의 합격률은 단순한 우연이 아니라, 실제로 통계적으로 유의한 차이가 있음.”

In [ ]:
# ==========================================================
# 🎯 01. 예시 데이터: 성공/실패 비율 비교 (표본 수 작음)
# ==========================================================
# [성공, 실패]
Group_A = [7, 3]   # A집단: 성공 7, 실패 3
Group_B = [2, 9]   # B집단: 성공 2, 실패 9

In [ ]:
# ==========================================================
# 📊 02. 기대빈도 확인
# ==========================================================
print("기대빈도(Expected Frequencies):")
print(stats.chi2_contingency([Group_A, Group_B])[3])

In [ ]:
# ==========================================================
# 🎯 03. Fisher’s Exact Test 수행
# ==========================================================
# 귀무가설(H₀): 두 그룹의 성공률은 같다.
# 대립가설(H₁): 두 그룹의 성공률은 다르다.

result_fisher = stats.fisher_exact([Group_A, Group_B])
print("Fisher’s Exact Test 결과:")
print(f"오즈비(odds ratio) = {result_fisher[0]:.3f}")
print(f"p-value = {result_fisher[1]:.4f}")

In [ ]:
# ==========================================================
# 🧩 04. 해석 가이드
# ==========================================================
# 오즈비(odds ratio) < 1 → A집단보다 B집단의 성공 확률이 낮음
# p-value < 0.05 → 두 그룹의 성공률이 유의하게 다름

### 📘 Chi-Square vs Fisher’s Exact 비교

| 구분 | Chi-Square Test | Fisher’s Exact Test |
|------|------------------|----------------------|
| **데이터 형태** | 범주형 (빈도표) | 범주형 (빈도표) |
| **표본 수** | 크거나 중간 이상 | 매우 작은 표본 (n<30) |
| **출력 값** | χ² 통계량, p-value | 오즈비, p-value |
| **가정** | 기대빈도 5 이상 | 기대빈도 작아도 OK |
| **주요 사용 예시** | 성별 vs 합격여부 | 약물효과 소규모 임상 실험 |

---

### ⚖️ 해석 가이드
| p-value | 해석 |
|----------|------|
| < 0.05 | 유의함 (변수 간 관계 존재) |
| ≥ 0.05 | 유의하지 않음 (독립적임) |

---

### 📚 실습 아이디어
- **Chi-Square:** 학과별 자격증 취득 여부 비교  
- **Fisher’s Exact:** 치료군 vs 대조군의 회복 여부 비교  
- **추가 심화:** 카이제곱 검정의 기대빈도 < 5인 셀이 많을 경우, 피셔 검정으로 대체

---

# 🧾 통계 검정 비교 요약

| 구분 | 데이터 형태 | 가정 | 대표 함수 |
|------|--------------|------|------------|
| **독립표본 t-검정** | 2개 그룹 평균 비교 | 정규성+등분산성 | `ttest_ind()` |
| **대응표본 t-검정** | 한 집단의 전후 비교 | 차이의 정규성 | `ttest_rel()` |
| **Mann–Whitney U** | 2개 독립집단 | 비모수 | `mannwhitneyu()` |
| **Wilcoxon** | 1개 집단 전후 비교 | 비모수 | `wilcoxon()` |
| **Chi-square / Fisher** | 범주형 변수 | 비연속형 분포 | `chi2_contingency()` / `fisher_exact()` |

---

🎯 **결론 (結)**  
데이터의 형태와 조건(정규성, 독립성, 표본 수)에 따라 **적절한 통계 검정법을 선택**해야 합니다.  
- “정규성 있음 → t-검정”  
- “정규성 없음 → 비모수 검정”  
- “범주형 데이터 → χ² 또는 Fisher’s Exact”  

👉 **결과를 숫자로 보는 것이 아니라, ‘차이가 의미 있는가’를 판단하는 것이 핵심입니다.**